# Medical Appointment No-Shows Analysis


## Introduction

In this notebook we will explore a dataset containing aproximately 100k medical appointments from the Brazilian public health system. One of the largest health system of the world.

> This is at municipal level combining the appointments occured in more then 45 units. As fasr as we know, the city has 20 million loss a year with no-shows. After so many studies happening in this data the average no-show now is 20\%. It is important to mention that all services offered to the municipality were free of charge.

This analysis is focused on the question of whether or not patients show up for their appointment, including a set of characterisitics about the patient in each row:

* `ScheduledDay`: tells us on which day the patient set up their appointment.
* `Neighborhood`: indicates the location of the hopital.
* `Scholarhip`: indicates whether or not the patient is enrolled in Brasilian welfare program called *Bolsa familia*
* `No-show`: it says `No` if the patient showed up to their appointment, and `Yes` if they didn't show up.

This analysis aims to outline some possible reasons for patient no-showing at the scheduled appointments, as well as get insights about the Brazilian public health system.

### Initial Statements

This section sets up import statements for all packages that will be used throught this python notebook.

In [1]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date

In [2]:
# set notebook environment
%matplotlib inline

sns.set_style('white')

%precision %.3f
pd.options.display.float_format = '{:,.2f}'.format
plt.rcParams["figure.figsize"] = 10,8
pd.set_option('display.width', 100)
plt.rcParams.update({'font.size': 22})

# Disable jedi autocompleter
%config Completer.use_jedi = False

In [3]:
df = pd.read_csv('../Datasets/KaggleV2-May-2016.csv.zip')

In [4]:
df.sample(5)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
30729,"947,292,296,889,981.00",5671684,F,2016-05-09T07:01:54Z,2016-05-13T00:00:00Z,28,CRUZAMENTO,1,0,0,0,0,0,Yes
84319,"381,291,323,955,179.00",5710285,F,2016-05-17T15:41:48Z,2016-06-02T00:00:00Z,18,PRAIA DO SUÁ,1,0,0,0,0,1,Yes
30045,"42,119,967,479,372.00",5732347,F,2016-05-24T11:21:00Z,2016-05-30T00:00:00Z,76,BOA VISTA,0,1,0,0,0,1,No
60857,"71,479,466,711,566.00",5691487,F,2016-05-12T12:56:43Z,2016-05-12T00:00:00Z,38,DO CABRAL,1,0,0,0,0,0,No
93648,"8,381,522,134,974.00",5773861,M,2016-06-06T07:34:50Z,2016-06-08T00:00:00Z,41,SÃO JOSÉ,0,0,0,0,0,0,Yes


### Getting to Know The Data

In [5]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 44.3 MB


In [6]:
print ("Rows     : " , df.shape[0])
print ("\nUnique values :  \n",df.nunique())

Rows     :  110527

Unique values :  
 PatientId          62299
AppointmentID     110527
Gender                 2
ScheduledDay      103549
AppointmentDay        27
Age                  104
Neighbourhood         81
Scholarship            2
Hipertension           2
Diabetes               2
Alcoholism             2
Handcap                5
SMS_received           2
No-show                2
dtype: int64


## Data Wrangling

### Update Data Types
Let's clean our dataset a bit.
We start by transforming the data the right format. It allows us to save some memory and simplifies further processing.

In [7]:
# Update neighborhood, Gender and No-show data types
# From object to category
df['Neighbourhood'] = df['Neighbourhood'].astype('category')
df['Gender'] = df['Gender'].astype('category')

# Transform Paient ID data type from float to int
df['PatientId'] = df['PatientId'].astype(int)

# Replace Yes/No with 1/0
df['No-show'].replace({'No': 0, 'Yes': 1}, inplace=True)

In [8]:
df[['ScheduledDay']].head(3)

,ScheduledDay
0,2016-04-29T18:38:08Z
1,2016-04-29T16:08:27Z
2,2016-04-29T16:19:04Z


While the scheduling date is recorded to the second, the time of the appointment is missing. Schade!
I would like to add two more columns to the dataset: number of waiting days as well as the appointment as day of the week. But before let's change the data type of the scheduling day and remove the time from it.

In [9]:
# From object to datetime
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'].str.split('T').str[0])
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'].str.split('T').str[0])

### Add, Remove and Update  Features

Now that we have converted the dates to the correct data type format: `Timestamps` , we can compute the waiting days till appointment by a simple substraction:

In [10]:
df['WaitingDays'] = df['AppointmentDay'] - df['ScheduledDay']

# Transform the waiting days to int
df['WaitingDays'] = df['WaitingDays'].dt.days

Remove appointment IDs, they don't contain any valuable information. We could also remove patient ID's, however I want to check if some patients didn't show more than twice

In [11]:
# remove the Appointment ID
df.drop('AppointmentID', axis=1, inplace=True);

In [12]:
# Rename columns
df.rename(columns={'Neighbourhood': 'Neighborhood',
                   'Handcap': 'Handicap',
                   'Hipertension': 'Hypertension'},
          inplace=True)

In [13]:
# change upper case to title case inside Neighborhood
df['Neighborhood'] = df['Neighborhood'].str.title()

'Jardim Da Penha'

### Data Consistency Check

In [11]:
# Let's check the consistency of the results
df[df['WaitingDays'] < 0]

,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,WaitingDays
27033,7839272661752,M,2016-05-10,2016-05-09,38,RESISTÊNCIA,0,0,0,0,1,0,1,-1
55226,7896293967868,F,2016-05-18,2016-05-17,19,SANTO ANTÔNIO,0,0,0,0,1,0,1,-1
64175,24252258389979,F,2016-05-05,2016-05-04,22,CONSOLAÇÃO,0,0,0,0,0,0,1,-1
71533,998231581612122,F,2016-05-11,2016-05-05,81,SANTO ANTÔNIO,0,0,0,0,0,0,1,-6
72362,3787481966821,M,2016-05-04,2016-05-03,7,TABUAZEIRO,0,0,0,0,0,0,1,-1


In [12]:
# remove negative waiting days
df = df[df['WaitingDays'] >= 0]

Why children didn't appear certainely depends on their parents. So here it doesn't make sense to consider children in the analysis. I'm, however, not sure about how it is in Brasil regarding age and doctor-patient confidentiality. My common sense tells me, that all patients under 15 should be ignored in the analysis.

In [13]:
len(df[df['Age'] < 15])

23257

We are going to drop over 23k observations. It seems a lot, but in my opinion it is the right thing to do. The rule is simple: the quality of your analysis, depends on the quality of your data!  
Now the question is, shouldn't we remove elderly people with handicaps? Maybe, but I prefer not to descriminate in this case.

In [14]:
df = df[df['Age'] > 14]
df.reindex();

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87265 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   PatientId       87265 non-null  int64         
 1   Gender          87265 non-null  category      
 2   ScheduledDay    87265 non-null  datetime64[ns]
 3   AppointmentDay  87265 non-null  datetime64[ns]
 4   Age             87265 non-null  int64         
 5   Neighbourhood   87265 non-null  category      
 6   Scholarship     87265 non-null  int64         
 7   Hipertension    87265 non-null  int64         
 8   Diabetes        87265 non-null  int64         
 9   Alcoholism      87265 non-null  int64         
 10  Handcap         87265 non-null  int64         
 11  SMS_received    87265 non-null  int64         
 12  No-show         87265 non-null  int64         
 13  WaitingDays     87265 non-null  int64         
dtypes: category(2), datetime64[ns](2), int64(10)
memory u